# Métodologia de Avaliação de Ferramentas AutoML

## 📊 Sistema de Pontuação

O modelo utiliza uma abordagem quantitativa padronizada para avaliar ferramentas de AutoML em cinco dimensões críticas:

1. **Descrição Funcional**
2. **Análise Estatística** 
3. **Transparência Algorítmica**
4. **Interpretabilidade**
5. **Análise Interna**

### 🔢 Métrica de Avaliação

$$
S = \begin{cases} 
0 & \text{if } N = 0 \\
1 & \text{if } N = 1 \\
2 & \text{if } N \geq 2 
\end{cases}
$$

Para cada critério de avaliação, é atribuído um valor **N** que representa o número de maneiras como a ferramenta atende ao requisito:

| Avaliação | Pontuação (Pᵢ) | Significado |
|-----------|----------------|-------------|
| N = 0     | 0 pontos       | Não atende ao critério |
| N = 1     | 1 ponto        | Atendimento parcial |
| N ≥ 2     | 2 pontos       | Atendimento total |

### 📈 Fórmula de Normalização

O score normalizado por categoria é calculado como:

$$
S = \left( \frac{\sum_{i=1}^{N} P_i}{N \cdot W} \right) \times 100
$$

Onde:
- **S**: Score normalizado (0-100%)
- **Pᵢ**: Pontuação da i-ésima pergunta (0, 1 ou 2)
- **N**: Número total de perguntas na categoria
- **W**: Pontuação máxima por pergunta (W=2)

## 🖥️ Como Usar o Modelo

1. **Seleção da Ferramenta**
   - Escolha a ferramenta no menu suspenso
   - Para novas ferramentas, digite o nome no campo "Outra"

2. **Processo de Avaliação**
   ```python
   Clique em "Iniciar Nova Avaliação"
   Para cada pergunta:
   - Selecione N=0 (Não atende)
   - Selecione N=1 (Parcial)
   - Selecione N≥2 (Total)
   Clique em "Calcular Scores"



# 📊 Visualização de Resultados

## Tabelas
- **Tabelas detalhadas** por critério
- **Gráficos comparativos** automáticos

---

## 📋 Saídas do Modelo

### 1. Tabela Resumo
Mostra por categoria:
- Score normalizado (%)
- Pontuação total
- Número de perguntas

### 2. Tabela Detalhada
Apresenta todas as avaliações individuais com:
- Critério avaliado
- Valor de N atribuído
- Pontuação calculada
- Score da categoria

### 3. Visualizações Gráficas

| Tipo de Gráfico | Utilidade | Exemplo |
|-----------------|-----------|---------|
| ![Barras](https://matplotlib.org/stable/_images/sphx_glr_bar_001.png) | Comparação direta entre categorias | `plt.bar()` |
| ![Radar](https://matplotlib.org/stable/_images/sphx_glr_radar_chart_001.png) | Visão holística do desempenho | `plt.polar()` |

---

## 📌 Considerações Metodológicas

✔ **Objetividade**: Pontuação baseada em critérios mensuráveis  
✔ **Normalização**: Comparação entre categorias com diferentes números de critérios  
✔ **Reprodutibilidade**: Aplicável a qualquer ferramenta AutoML  
✔ **Abordagem Quantitativa**: Reduz subjetividade na avaliação  

---

## 🔄 Fluxo de Trabalho Recomendado

1. Avalie todas as ferramentas de interesse
2. Consulte a análise comparativa
3. Identifique pontos fortes e fracos
4. Repita para novas versões

> **Nota**: Os resultados são salvos automaticamente permitindo:
> - Análises temporais
> - Consolidação de dados
> - Comparação histórica

In [20]:
%%javascript
// Mantém os outputs visíveis
Jupyter.notebook.get_cells().forEach(function(cell) {
    if (cell.cell_type === 'code') {
        cell.output_area.outputs = cell.output_area.outputs || [];
    }
});

import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import pi
import pickle
import os

# Configurações de estilo
plt.style.use('seaborn')
primary_color = '#1f77b4'
palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

## Dados das Perguntas por Categoria
questions = {
    "Descrição Funcional": [
        "É possível identificar quais são as etapas do pipeline?",
        "É possível identificar os componentes utilizados em cada etapa?",
        "Está claro quais são as entradas e saídas?",
        "Está claro como o processo é executado na prática?"
    ],
    "Análise Estatística": [
        "As previsões do modelo são apresentadas para as classes (malwares e benignos)?",
        "Mantem o histórico dos dados utilizados para treinar e testar o modelo?",
        "Na seleção de características é informado ao usuario quais são as mais relevantes?",
        "As métricas de desempenho do modelo são apresentadas?",
        "Existem formas de visualizar os resultados (gráficos, tabelas)?"
    ],
    "Transparência Algorítmica": [
        "É possível identificar quais modelos são utilizados?",
        "É possivel identificar avisos, informações e erros registrados nos logs do sistema?",
        "É possível identificar os hiperparametros dos modelos gerados?",
        "É possível identificar se os dados são balanceados?"
    ],
    "Interpretabilidade": [
        "O motivo da redução da dimensionalidade é explicado? (Pré-Modelo)",
        "Existem técnicas específicas de redução de dimensionalidade para o domínio? (Pré-Modelo)",
        "Há métodos de interpretabilidade global disponíveis?",
        "O framework oferece modelos com interpretabilidade intrínseca? (In-Modelo)",
        "É possível avaliar a diferença entre previsões e valores reais?",
        "Há métodos de interpretabilidade independentes do modelo? (Pós-Modelo)"
    ],
    "Análise Interna": [
        "Existem métodos para visualizar a estrutura do modelo? (Pós-Modelo)"
    ]
}

# Arquivo para salvar os resultados
RESULTS_FILE = 'automl_evaluations.pkl'

class AutoMLComparativeEvaluation:
    def __init__(self):
        self.all_results = pd.DataFrame()
        self.load_existing_data()
    
    def load_existing_data(self):
        if os.path.exists(RESULTS_FILE):
            with open(RESULTS_FILE, 'rb') as f:
                self.all_results = pickle.load(f)
    
    def save_results(self):
        with open(RESULTS_FILE, 'wb') as f:
            pickle.dump(self.all_results, f)
    
    def add_evaluation(self, tool_name, results_df):
        results_df['Ferramenta'] = tool_name
        self.all_results = pd.concat([self.all_results, results_df], ignore_index=True)
        self.save_results()
    
    def get_comparison_data(self):
        if self.all_results.empty:
            return pd.DataFrame()
        
        comparison_df = self.all_results.groupby(['Ferramenta', 'Categoria']).agg({
            'Pontuação': 'sum',
            'Score': 'first'
        }).reset_index()
        
        pivot_df = comparison_df.pivot(index='Ferramenta', columns='Categoria', values='Score')
        total_scores = self.all_results.groupby('Ferramenta')['Pontuação'].sum()
        pivot_df['Pontuação Total'] = total_scores
        
        return pivot_df.sort_values('Pontuação Total', ascending=False)
    
    def plot_comparison_radar(self):
        comparison_data = self.get_comparison_data()
        if comparison_data.empty:
            print("Nenhum dado disponível para comparação")
            return
        
        categories = list(questions.keys())
        N = len(categories)
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]
        
        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
        
        tools = comparison_data.index.tolist()
        for idx, tool in enumerate(tools):
            values = comparison_data.loc[tool, categories].values.flatten().tolist()
            values += values[:1]
            ax.plot(angles, values, color=palette[idx % len(palette)], linewidth=2, 
                    linestyle='solid', label=f"{tool} ({int(comparison_data.loc[tool, 'Pontuação Total'])} pts)")
            ax.fill(angles, values, color=palette[idx % len(palette)], alpha=0.1)
        
        plt.xticks(angles[:-1], categories, color='grey', size=12)
        ax.set_rlabel_position(30)
        plt.yticks([20, 40, 60, 80, 100], ["20%", "40%", "60%", "80%", "100%"], color="grey", size=10)
        plt.ylim(0, 100)
        
        plt.title("Comparação entre Ferramentas AutoML\n(Score Normalizado por Categoria)", size=15, y=1.1)
        plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
        plt.show()
    
    def plot_comparison_bars(self):
        comparison_data = self.get_comparison_data()
        if comparison_data.empty:
            print("Nenhum dado disponível para comparação")
            return
        
        categories = list(questions.keys())
        tools = comparison_data.index.tolist()
        bar_width = 0.15
        index = np.arange(len(categories))
        
        plt.figure(figsize=(15, 8))
        
        for i, tool in enumerate(tools):
            values = comparison_data.loc[tool, categories].values
            plt.bar(index + i*bar_width, values, bar_width, 
                    color=palette[i % len(palette)], label=f"{tool} ({int(comparison_data.loc[tool, 'Pontuação Total'])} pts)")
        
        plt.xlabel('Categorias')
        plt.ylabel('Score Normalizado (%)')
        plt.title('Comparação de Ferramentas por Categoria')
        plt.xticks(index + bar_width*(len(tools)-1)/2, categories, rotation=45)
        plt.ylim(0, 110)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

class ToolEvaluator:
    def __init__(self, comparative_system, tool_name):
        self.comparative_system = comparative_system
        self.tool_name = tool_name
        self.results = pd.DataFrame(columns=['Categoria', 'Pergunta', 'N', 'Pontuação', 'Score'])
        self.W = 2
        
        self.create_widgets()
    
    def create_widgets(self):
        self.widgets = {}
        
        for category, q_list in questions.items():
            for question in q_list:
                key = f"{category} | {question}"
                self.widgets[key] = widgets.Dropdown(
                    options=[
                        ('Não atende (N=0)', 0),
                        ('Parcial (N=1)', 1),
                        ('Total (N≥2)', 2)
                    ],
                    description=question,
                    style={'description_width': 'initial'},
                    layout={'width': '800px'}
                )
        
        self.submit_button = widgets.Button(
            description="Salvar Avaliação",
            button_style='success',
            icon='save'
        )
        self.submit_button.on_click(self.save_evaluation)
    
    def display_interface(self):
        display(widgets.HTML(f"<h2 style='color:{primary_color};'>Avaliação: {self.tool_name}</h2>"))
        
        accordion = widgets.Accordion(children=[
            widgets.VBox([self.widgets[key] for key in self.widgets if key.startswith(category)])
            for category in questions.keys()
        ])
        
        for i, category in enumerate(questions.keys()):
            accordion.set_title(i, f"{category} ({len(questions[category])} perguntas)")
        
        display(accordion)
        display(self.submit_button)
    
    def save_evaluation(self, b):
        self.results = pd.DataFrame(columns=['Categoria', 'Pergunta', 'N', 'Pontuação', 'Score'])
        
        for key, widget in self.widgets.items():
            category, question = key.split(" | ")
            N = widget.value
            points = min(N, 2)
            
            self.results.loc[len(self.results)] = {
                'Categoria': category,
                'Pergunta': question,
                'N': N,
                'Pontuação': points,
                'Score': None
            }
        
        for category in questions.keys():
            cat_data = self.results[self.results['Categoria'] == category]
            sum_Pi = cat_data['Pontuação'].sum()
            N_questions = len(cat_data)
            
            S = (sum_Pi / (N_questions * self.W)) * 100 if N_questions > 0 else 0
            
            self.results.loc[self.results['Categoria'] == category, 'Score'] = S
        
        self.comparative_system.add_evaluation(self.tool_name, self.results)
        
        clear_output()
        display(widgets.HTML(f"<div style='color:green; font-weight:bold;'>Avaliação para {self.tool_name} salva com sucesso!</div>"))
        self.display_results()
    
    def display_results(self):
        display(widgets.HTML(f"<h3>Resumo para {self.tool_name}</h3>"))
        
        summary = self.results.groupby('Categoria').agg({
            'Pontuação': 'sum',
            'Score': 'first'
        })
        summary['Perguntas'] = self.results.groupby('Categoria').size()
        display(summary)
        
        display(widgets.HTML(f"<p><b>Pontuação Total:</b> {self.results['Pontuação'].sum()} pontos</p>"))

class AutoMLEvaluationUI:
    def __init__(self):
        self.comparative_system = AutoMLComparativeEvaluation()
        self.create_ui()
    
    def create_ui(self):
        self.tool_selector = widgets.Dropdown(
            options=['AutoSklearn', 'AutoGloun', 'TPOT', 'LinghtAutoML', 'MlJar', 
                    'AutoPyTorch', 'HyperGBM', 'MH-AutoML', 'Nova Ferramenta'],
            description='Ferramenta:',
            style={'description_width': 'initial'}
        )
        
        self.new_tool_text = widgets.Text(
            placeholder="Digite o nome de uma nova ferramenta",
            description='Outra:',
            disabled=False
        )
        
        self.start_button = widgets.Button(
            description="Iniciar Avaliação",
            button_style='primary',
            icon='play'
        )
        
        self.compare_button = widgets.Button(
            description="Comparar Ferramentas",
            button_style='info',
            icon='bar-chart'
        )
        
        self.output = widgets.Output()
        
        self.start_button.on_click(self.start_evaluation)
        self.compare_button.on_click(self.show_comparison)
        
        tools_box = widgets.HBox([self.tool_selector, self.new_tool_text])
        buttons_box = widgets.HBox([self.start_button, self.compare_button])
        
        self.ui = widgets.VBox([
            widgets.HTML("<h1 style='text-align: center; color: #1f77b4;'>Sistema de Avaliação Comparativa de AutoML</h1>"),
            widgets.HTML("""
            <div style='background-color: #f7f7f7; padding: 15px; border-radius: 5px; margin-bottom: 20px;'>
                <h3 style='color: #333;'>Como usar:</h3>
                <ol>
                    <li>Selecione uma ferramenta ou digite o nome de uma nova</li>
                    <li>Clique em <b>Iniciar Avaliação</b> para avaliar a ferramenta</li>
                    <li>Para cada critério, indique <b>N</b> (número de maneiras que a ferramenta atende):</li>
                    <ul>
                        <li><b>N=0</b>: Não atende (0 pontos)</li>
                        <li><b>N=1</b>: Atende parcialmente (1 ponto)</li>
                        <li><b>N≥2</b>: Atende totalmente (2 pontos)</li>
                    </ul>
                    <li>Clique em <b>Comparar Ferramentas</b> para ver gráficos comparativos</li>
                </ol>
            </div>
            """),
            tools_box,
            buttons_box,
            self.output
        ])
    
    def display(self):
        display(self.ui)
    
    def start_evaluation(self, b):
        with self.output:
            clear_output()
            tool_name = self.new_tool_text.value if self.new_tool_text.value else self.tool_selector.value
            if not tool_name:
                print("Por favor, selecione ou digite o nome de uma ferramenta")
                return
                
            evaluator = ToolEvaluator(self.comparative_system, tool_name)
            evaluator.display_interface()
    
    def show_comparison(self, b):
        with self.output:
            clear_output()
            if self.comparative_system.all_results.empty:
                print("Nenhuma ferramenta avaliada ainda. Por favor, avalie pelo menos uma ferramenta.")
                return
                
            display(widgets.HTML("<h2 style='color:#1f77b4;'>Comparação entre Ferramentas</h2>"))
            
            comparison_data = self.comparative_system.get_comparison_data()
            display(comparison_data)
            
            self.comparative_system.plot_comparison_radar()
            self.comparative_system.plot_comparison_bars()



<IPython.core.display.Javascript object>

In [22]:
# Iniciar a interface
ui = AutoMLEvaluationUI()
ui.display()